In [70]:
import time
import re

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import requests
import time
import random

Part I: Website Scrapy

In [2]:
service = Service(ChromeDriverManager().install())

In [3]:
DRIVER = webdriver.Chrome(service = service)

In [20]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search

In [7]:
pip install requests beautifulsoup4 google

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 508.4 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [55]:
def scrape_website(url):
    response = requests.get(url, timeout = 10)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('title').text
    paragraphs = [p.text for p in soup.find_all('p')]

    return url, title, paragraphs

In [56]:
# test the function
search_query = "Seattle Seahawks"
links = scrape_page(search_query)
links

['https://maps.google.com/maps%3Fq%3DSeattle%2BSeahawks%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111',
 'https://www.fieldgulls.com/2024/2/27/24084256/teds-talk-calais-campbell-mike-macdonald-reunion-seahawks-nfl-analysis',
 'https://sports.mynorthwest.com/1773109/seattle-seahawks-roster-offseason-nfl-salary-cap/',
 'https://www.fantasypros.com/nfl/news/434969/jamal-adams-seahawks-could-move-on-this-offseason.php',
 'https://www.fieldgulls.com/2024/2/27/24085345/2024-seattle-seahawks-nfl-offseason-roster-tracker-latest-signings-trades-salary-cap-casualties',
 'https://www.fantasypros.com/nfl/news/434968/geno-smith-veteran-is-seahawks-qb-until-he-isnt-per-john-schneider.php',
 'https://www.seattletimes.com/sports/seahawks/is-there-a-russell-wilson-redemption-arc-coming/',
 'https://www.cbssports.com/nfl/news/former-seahawks-star-richard-sherman-arrested-for-suspicion-of-dui-in-washington/',
 'https://12thmanrising.com/posts/proposed-trade-remakes-seahawks-defensive-line-best-nfl

In [57]:
df = pd.DataFrame({'link': links})
df.to_csv('links.csv')

In [68]:
def scrape_page_content(url):
    result = {'URL': url, 'Title': '', 'Paragraphs': '', 'Processing Time': '', 'Error': ''}
    
    try:
        start_time = time.time()
        print(f"Processing URL: {url}")

        response = requests.get(url, timeout=10, allow_redirects=True)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        title_tag = soup.find('title')
        if title_tag:
            result['Title'] = title_tag.text
        else:
            result['Title'] = "Title not found on the page"

        paragraphs = [p.text for p in soup.find_all('p')]
        result['Paragraphs'] = paragraphs

        end_time = time.time()
        result['Processing Time'] = end_time - start_time
        print(f"Processing time: {result['Processing Time']} seconds")

    except requests.exceptions.RequestException as e:
        print(f"Error scraping content from {url}: {e}")
        result['Error'] = str(e)

    return result

# Collect results in a list
results = [scrape_page_content(url) for url in links]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(results)

# Display the DataFrame
print(df)


Processing URL: https://maps.google.com/maps%3Fq%3DSeattle%2BSeahawks%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111
Error scraping content from https://maps.google.com/maps%3Fq%3DSeattle%2BSeahawks%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111: 404 Client Error: Not Found for url: https://maps.google.com/maps%3Fq%3DSeattle%2BSeahawks%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111
Processing URL: https://www.fieldgulls.com/2024/2/27/24084256/teds-talk-calais-campbell-mike-macdonald-reunion-seahawks-nfl-analysis
Processing time: 1.9950439929962158 seconds
Processing URL: https://sports.mynorthwest.com/1773109/seattle-seahawks-roster-offseason-nfl-salary-cap/
Error scraping content from https://sports.mynorthwest.com/1773109/seattle-seahawks-roster-offseason-nfl-salary-cap/: 403 Client Error: Forbidden for url: https://sports.mynorthwest.com/1773109/seattle-seahawks-roster-offseason-nfl-salary-cap/
Processing URL: https://www.fantasypros.com/nfl/news/434969/jamal-adams

In [72]:
def clean_paragraph(paragraph_list):

    paragraph = ' '.join(paragraph_list)
    # Remove website links
    paragraph = re.sub(r'https?://\S+', '', paragraph)
    
    # Remove special characters
    paragraph = re.sub(r'[^a-zA-Z0-9\s]', '', paragraph)
    
    # Remove extra whitespaces
    paragraph = ' '.join(paragraph.split())
    
    return paragraph

# Apply the cleaning function to the 'Paragraphs' column
df['Paragraphs'] = df['Paragraphs'].apply(clean_paragraph)

# Remove rows with empty paragraphs
df = df[df['Paragraphs'].astype(bool)]

# Display the cleaned dataframe
print(df)

                                                   URL  \
1    https://www.fieldgulls.com/2024/2/27/24084256/...   
3    https://www.fantasypros.com/nfl/news/434969/ja...   
4    https://www.fieldgulls.com/2024/2/27/24085345/...   
5    https://www.fantasypros.com/nfl/news/434968/ge...   
7    https://www.cbssports.com/nfl/news/former-seah...   
..                                                 ...   
259  https://support.google.com/websearch%3Fp%3Dws_...   
264  https://www.patriots.com/video/tom-brady-s-200...   
265  https://www.patriots.com/video/tom-brady-s-200...   
270  https://support.google.com/websearch%3Fp%3Dws_...   
279  https://support.google.com/websearch%3Fp%3Dws_...   

                                                 Title  \
1    Ted’s Talk: A Calais Campbell, Mike Macdonald ...   
3    Jamal Adams: Seahawks could move on this offse...   
4    2024 Seattle Seahawks offseason roster tracker...   
5    Geno Smith: Veteran is the Seahawks QB 'until ...   
7    Former S

In [73]:
df.to_csv('content.csv')